In [3]:
import pygame,sys
from pygame.locals import *
import time
pygame.init()

FPS = 30
clock = pygame.time.Clock()

window_l = 500
window_w = 300
DISPLAYSURF = pygame.display.set_mode((window_l,window_w),0,32)
pygame.display.set_caption('Kill the blob!')

BLACK = (0,0,0)
WHITE = (255,255,255)
#img = pygame.image.load('Art//idle_r.png')
img_r = pygame.image.load('Art//idle_r.png')
#img_r = pygame.transform.rotozoom(img_r,0,2)
img_l = pygame.image.load('Art//idle_l.png')
#img_l = pygame.transform.rotozoom(img_l,0,2)
w_l = pygame.image.load('Art//walk_l.png')
#w_l = pygame.transform.rotozoom(w_l,0,2)
w_r = pygame.image.load('Art//walk_r.png')
#w_r = pygame.transform.rotozoom(w_r,0,2)
blb_l = pygame.image.load('Art//blob_l.png')
blb_r = pygame.image.load('Art//blob_r.png')
xpos = window_l/2
ypos = window_w-57


l_att = pygame.image.load('Art//attack_left.png')
#l_att = pygame.transform.rotozoom(l_att,0,2)
r_att = pygame.image.load('Art//attack_right.png')
#r_att = pygame.transform.rotozoom(r_att,0,2)

class Player:
    def __init__(self):
        self.xpos = xpos
        self.ypos = ypos
        self.img = img_r
        self.facingRight = True
        self.facingLeft = False
        self.ToObjRight = False
        self.ToObjLeft = True
        self.Flag=0
        
    def walk(self,press):
        if press[K_LEFT] & self.facingRight== True:
            self.facingLeft=True
            self.facingRight=False
            self.img = img_l
        elif press[K_LEFT] & self.facingLeft== True:
            if self.Flag==0:
                self.img = w_l
                self.xpos = self.xpos-1
                self.Flag=1
                return
            elif self.Flag == 1:
                self.img = img_l
                self.xpos = self.xpos-1
                self.Flag=0
                return
        elif press[K_RIGHT] & self.facingRight== True:
            if self.Flag==0:
                self.img = w_r
                self.xpos = self.xpos+1
                self.Flag=1
                return
            elif self.Flag == 1:
                self.img = img_r
                self.xpos = self.xpos+1
                self.Flag=0
                return
        elif press[K_RIGHT] & self.facingLeft== True:
            self.facingLeft=False
            self.facingRight=True
            self.img = img_r
            
#    def jump(self,press):
#        if press[K_UP]:
#            for i in range(1000):
#                if i%500==0:
#                    self.ypos = self.ypos-1
#                    time.sleep(0.1)
#                    refresh()
#                    break
#            for i in range(1000):
#                if i%500==0:
#                    self.ypos = self.ypos+1
#                    time.sleep(0.1)
#                    refresh()
#                    break
#        else:
#            return
    
    def attack(self,press):
        if press[K_SPACE]:
            if self.facingRight==True & self.ToObjLeft==True:
                self.img = r_att
                check_attack(self,blob)
            elif self.facingLeft==True & self.ToObjRight==True:
                self.img = l_att
                check_attack(self,blob)
        else:
            return
        
    
    
class blob:
    def __init__(self):
        self.xpos = 300
        self.ypos = 194
        self.img = blb_l
        self.facingLeft = True
        self.facingRight = False
    def change_pos(self,ToObjRight,ToObjLeft):
        if ToObjLeft == True:
            self.img = blb_l
            self.facingLeft = True
            self.facingRight = False
        elif ToObjRight == True:
            self.img = blb_r
            self.facingLeft = False
            self.facingRight = True

    

def check_attack(Player,blob):
    if Player.ToObjLeft==True & ((Player.xpos+50)>=blob.xpos):
        blob.xpos = 700
    if Player.ToObjRight==True & (Player.xpos<=(blob.xpos+100)):
        blob.xpos = 700
def refresh():
    DISPLAYSURF.fill(WHITE)
    pd = DISPLAYSURF.blit(Player.img,(Player.xpos,Player.ypos))
    bd = DISPLAYSURF.blit(blob.img,(blob.xpos,blob.ypos))
    if pd.right < bd.right:
        Player.ToObjLeft = True
        Player.ToObjRight = False
    elif pd.left > bd.left:
        Player.ToObjLeft = False
        Player.ToObjRight = True
    blob.change_pos(Player.ToObjRight,Player.ToObjLeft)
    
run = True
Player = Player()
blob = blob()
#pygame.key.set_repeat(10,10)
while(run):
    refresh()
    
    press = pygame.key.get_pressed()
    Player.walk(press)
    Player.attack(press)
    
    pygame.time.delay(10)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    
        pygame.display.update()
        clock.tick(FPS)

SystemExit: 

C:\Users\Joy Parikh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
# Making an MCTS and experimenting with it here
from collections import defaultdict
import math
class MCTS:
    def __init__(self,next_no=1):
        self.q = defaultdict(int)
        self.n = defaultdict(int)
        self.child = dict()
        self.next_no = next_no
        
    def traverse(self,node):
        if node not in self.child:
            return node.random_child()
        score = dict()
        
        for i in self.child[node]:
            if self.n[i] == 0:
                score[i]=(float("-inf"))
            else:
                score[i] = self.q[i]/self.n[i]
        return list(self.child.keys())[list(score.values()).index(max(score.values()))]
    def roll_out(self,node):
        path = self._select(node)
        #print(path)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backprop(path,reward)
#the standard methods for tree search begins below. Select, Expand, Simulate and Backpropagate

    def _select(self,node):
        path=[]
        while True:
            path.append(node)
            if node not in self.child or not self.child[node]:
                #print('huthut')
                return path
            unexplored = self.child[node] - self.child.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return(path)
            node = self._ucb(node)
            
    def _expand(self,node):
        if node in self.child:
            return
        self.child[node] = node.find_child()
        
    def _simulate(self,node):
        reward_inv = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1-reward if reward_inv else reward
            node = node.random_child()
            reward_inv = not reward_inv
            #print('stuck in loop with ',reward_inv)
    def _backprop(self,path,reward):
        for node in reversed(path):
            self.n[node] +=1
            self.q[node] +=reward
            reward = 1-reward
            
    def _ucb(self,node):
        assert all(n in self.child for n in self.child[node])
        
        log_n = math.log(self.n[node])
        ucb=dict()
        for i in self.child[node]:
            ucb[i] = (self.q[i] / self.n[i] + self.next_no * math.sqrt(log_n/self.n[i]))
        
        return list(self.child.keys())[list(ucb.values()).index(max(ucb.values()))]

In [2]:
from abc import ABC,abstractmethod

class Node(ABC):
    
    @abstractmethod
    def find_child(self):
        return set()
    
    @abstractmethod
    def random_child(self):
        return None
    
    @abstractmethod
    def is_terminal(self):
        return True
    
    @abstractmethod
    def reward(self):
        return 0
    
    @abstractmethod
    def __hash__(self):
        return 123456789
    
    @abstractmethod
    def __isEq__(node1,node2):
        return True

In [ ]:

"""
An example implementation of the abstract Node class for use in MCTS
If you run this file then you can play against the computer.
A tic-tac-toe board is represented as a tuple of 9 values, each either None,
True, or False, respectively meaning 'empty', 'X', and 'O'.
The board is indexed by row:
0 1 2
3 4 5
6 7 8
For example, this game board
O - X
O X -
X - -
corrresponds to this tuple:
(False, None, True, False, True, None, True, None, None)
"""

from collections import namedtuple
from random import choice

_TTTB = namedtuple("TicTacToeBoard", "tup turn winner terminal")

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class TicTacToeBoard(_TTTB, Node):
    def find_child(board):
        if board.terminal:  # If the game is finished then no moves can be made
            return set()
        # Otherwise, you can make a move in each of the empty spots
        #print('finding children')
        return {
            board.make_move(i) for i, value in enumerate(board.tup) if value is None
        }

    def random_child(board):
        if board.terminal:
            return None  # If the game is finished then no moves can be made
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    def reward(board):
        if not board.terminal:
            raise RuntimeError(f"reward called on nonterminal board {board}")
        if board.winner is board.turn:
            # It's your turn and you've already won. Should be impossible.
            raise RuntimeError(f"reward called on unreachable board {board}")
        if board.turn is (not board.winner):
            return 0  # Your opponent has just won. Bad.
        if board.winner is None:
            return 0.5  # Board is a tie
        # The winner is neither True, False, nor None
        raise RuntimeError(f"board has unknown winner type {board.winner}")

    def is_terminal(board):
        return board.terminal

    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1 :]
        turn = not board.turn
        winner = _find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        #print('returning from making a move at',index)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    def to_pretty_string(board):
        to_char = lambda v: ("X" if v is True else ("O" if v is False else " "))
        rows = [
            [to_char(board.tup[3 * row + col]) for col in range(3)] for row in range(3)
        ]
        return (
            "\n  1 2 3\n"
            + "\n".join(str(i + 1) + " " + " ".join(row) for i, row in enumerate(rows))
            + "\n"
        )


def play_game():
    tree = MCTS()
    board = new_tic_tac_toe_board()
    print(board.to_pretty_string())
    while True:
        row_col = input("enter row,col: ")
        row, col = map(int, row_col.split(","))
        index = 3 * (row - 1) + (col - 1)
        if board.tup[index] is not None:
            raise RuntimeError("Invalid move")
        board = board.make_move(index)
        print(board.to_pretty_string())
        if board.terminal:
            break
        # You can train as you go, or only at the beginning.
        # Here, we train as we go, doing fifty rollouts each turn.
        for _ in range(50):
            tree.roll_out(board)
        board = tree.traverse(board)
        print(board.to_pretty_string())
        if board.terminal:
            break


def _winning_combos():
    for start in range(0, 9, 3):  # three in a row
        yield (start, start + 1, start + 2)
    for start in range(3):  # three in a column
        yield (start, start + 3, start + 6)
    yield (0, 4, 8)  # down-right diagonal
    yield (2, 4, 6)  # down-left diagonal


def _find_winner(tup):
    "Returns None if no winner, True if X wins, False if O wins"
    for i1, i2, i3 in _winning_combos():
        v1, v2, v3 = tup[i1], tup[i2], tup[i3]
        if False is v1 is v2 is v3:
            return False
        if True is v1 is v2 is v3:
            return True
    return None


def new_tic_tac_toe_board():
    return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)


if __name__ == "__main__":
    play_game()


  1 2 3
1      
2      
3      

enter row,col: 2,2

  1 2 3
1      
2   X  
3      



In [80]:
a = dict()
a[1] = 2
a[3]=5
a[2]=7
list(a.keys())[list(a.values()).index(max(a.values()))]

2